In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from util.ZigZag import ZigZag
from util.BuildData import BuildData4
from util.CalcReturns import CalcReturns

from sklearn import svm, preprocessing

In [2]:
df = pd.read_csv('EURUSD1.csv', header=0)
train = np.array(df.Close)

In [3]:
Z = ZigZag(train, 0.004)
R = CalcReturns(train)
S = BuildData4(Z, R, 61)

t_count = 1024
dims = S["data"].shape[1]

D = S["data"]
L = []

for i in S["label"]:
    if i == 1:
        L.append([0, 1])
    elif i == -1:
        L.append([1, 0])

L = np.array(L)
        
#print D.shape
#print L.shape


In [4]:
def neural_network_model(x):

    hl1 = {'w' : tf.Variable(tf.random_normal([dims, n_nodes_hl1])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hl2 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hl3 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    hl4 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl3, n_nodes_hl4])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl4]))}

    out = {'w' : tf.Variable(tf.random_normal([n_nodes_hl4, n_classes])),
           'b' : tf.Variable(tf.random_normal([n_classes]))}

    l1 = tf.matmul(x, hl1['w']) + hl1['b']
    l1 = tf.nn.relu(l1)

    l2 = tf.matmul(l1, hl2['w']) + hl2['b']
    l2 = tf.nn.relu(l2)

    l3 = tf.matmul(l2, hl3['w']) + hl3['b']
    l3 = tf.nn.relu(l3)
    
    l4 = tf.matmul(l3, hl4['w']) + hl4['b']
    l4 = tf.tanh(l4)

    y = tf.add(tf.matmul(l2, out['w']), out['b'], name='nn')
    return y

In [5]:
batch_size = 100

def train_and_save_nn(x,y):
    nn = neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = nn, labels = y))
    optimizer = tf.train.AdamOptimizer(0.006).minimize(cost)   
    
    correct = tf.equal(tf.argmax(nn,1), tf.argmax(y_train, 1), name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name = 'accuracy')
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int((len(D) - t_count)/batch_size)):
                
                epoch_x = D[i*batch_size : (i+1)*batch_size]
                epoch_y = L[i*batch_size : (i+1)*batch_size]
                
                _, c = session.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
                
        saver.save(session, 'data/fxnn.ckpt')
        
        print('Last epoch loss: ', epoch_loss)
        return nn, correct, accuracy

In [6]:
def test_nn(x_test, y_test):
    with tf.Session() as session:
        saver = tf.train.import_meta_graph('data/fxnn.ckpt.meta')
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph    = tf.get_default_graph()
        correct  = graph.get_tensor_by_name('correct:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        
        x = graph.get_tensor_by_name('x:0')
        y = graph.get_tensor_by_name('y:0')

        r = tf.cast(correct, 'float').eval({x:x_test, y:y_test})       

        #for i in range(len(r)):
        #    print '%s %s' % (r[i], y_test[i])        

        test_buy  = 0.0
        test_sell = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0

        for i in range(len(r)):
            if y_test[i][1] == 1:
                test_buy += 1.0;
            elif y_test[i][0] == 1:
                test_sell += 1.0;

            if r[i] == 1:
                if y_test[i][1] == 1:
                    correct_buy += 1.0;
                elif y_test[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if y_test[i][1] == 1:
                    incorrect_buy += 1.0;
                elif y_test[i][0] == 1:
                    incorrect_sell += 1.0;

        print('Accuracy:',accuracy.eval({x:x_test, y:y_test}))

        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )

In [7]:
n_nodes_hl1 = 100
n_nodes_hl2 = 100
n_nodes_hl3 = 100
n_nodes_hl4 = 100

n_classes = 2
hm_epochs = 1000

x_train = tf.placeholder('float', [None, dims], name='x')
y_train = tf.placeholder('float', name='y')

train_and_save_nn(x_train, y_train)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 1000, 'loss', 764.93957543375723)
('Epoch', 100, 'completed out of', 1000, 'loss', 73.885450065135956)
('Epoch', 200, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 300, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 400, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 500, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 600, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 700, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 800, 'completed out of', 1000, 'loss', 73.885487228631973)
('Epoch', 900, 'completed out of', 1000, 'loss', 73.885487228631973)
('Last epoch loss: ', 73.885487228631973)


(<tf.Tensor 'nn:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'correct:0' shape=<unknown> dtype=bool>,
 <tf.Tensor 'accuracy:0' shape=<unknown> dtype=float32>)

In [8]:
x_test = D[len(D)-t_count:]
y_test = L[len(L)-t_count:]

test_nn(x_test, y_test)


('Accuracy:', 0.52539062)
('Test buy:  ', 486.0)
('Test sell: ', 538.0)
('Correct buy:  ', 0.0, 'Incorrect buy:  ', 486.0, 'Accuracy: ', 0.0)
('Correct sell: ', 538.0, 'Incorrect sell: ', 0.0, 'Accuracy: ', 100.0)


In [10]:
#print x_test
#print y_test

#t_dims = 12
#x0 = np.array(S["data"][t_dims]).reshape(1,-1)
#y0 = L[t_dims]

#print x0.shape
#print y0.shape

with tf.Session() as session:
        saver = tf.train.import_meta_graph('data/fxnn.ckpt.meta')
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph = tf.get_default_graph()  
                
        x  = graph.get_tensor_by_name('x:0')
        y  = graph.get_tensor_by_name('y:0')
        nn = graph.get_tensor_by_name('nn:0')
        
        #correct  = graph.get_tensor_by_name('correct:0')
        #accuracy = graph.get_tensor_by_name('accuracy:0')        
        
        for i in range(t_count):
            t = np.array(x_test[i]).reshape(1,-1)            
            u = y_test[i]
            
            print '%s %s' % (session.run(tf.argmax(nn,1), feed_dict = {x:t}), u)
            


[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[1] [1 0]
[1] [1 0]
[1] [1 0]
[1] [1 0]
[1] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[1] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[1] [1 0]
[1] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[1] [1 0]
[1] [1 0]
[1] [1 0]
[1] [1 0]
[1] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
[0] [1 0]
